In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 75%; }
</style>
"""))

In [2]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
from src.learning.models import create_nvidia_cnn, create_mlp, \
                                create_standalone_nvidia_cnn, create_standalone_resnet, \
                                create_multi_model_2, create_multi_model_3, create_multi_cnn

In [11]:
def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Epoch')
    x = np.arange(0, len(plot_elems[0]['data']), 1)
    
    for plot_elem in plot_elems:
        plt.errorbar(x, plot_elem['data'], yerr=plot_elem['error'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o', capsize=5)

    plt.grid(axis='x')
    plt.legend(loc='best', prop={'size': 15})
    plt.show()
    plt.savefig('./' + title + '.png')
    
def get_model_num(model_path, model_prefix):
    model_files = [fn for fn in os.listdir(model_path) if fn.startswith(model_prefix) and fn.endswith('.h5') and 'dagger_' not in fn]
    # expected format is "model_n1_m1_9.h5"
    existing_nums = [int(fn.split('_')[3].split('.')[0]) for fn in model_files]
    
    if len(existing_nums) == 0:
        return 1
    else:
        latest_num = sorted(existing_nums)[-1]
        return int(latest_num) + 1

In [6]:
config_manager = ConfigurationManager()
config = config_manager.config

memory_variants = [(1, 1), (2, 1), (2, 2), (4, 1), (4, 2), (6, 1), (6, 2)]
memory_variants = memory_variants[1:]
memory = memory_variants[0]

model_path = '../../training/models/'

In [ ]:
# Model experiments

epochs = 6
batch_size = 32
verbose = 1

losses = []
val_losses = []

generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer')
frame_shape, numeric_shape, diff_shape = generator.get_shapes()
tqdm.write('Target shape: {}'.format(diff_shape))
tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))

models = []

#mlp = create_mlp(input_shape=numeric_shape)
#cnn = create_cnn_nvidia(input_shape=frame_shape)
#models.append((create_multi_model_3(mlp, cnn, output_shape=diff_shape), generator.generate_with_numeric))

for model, generate_method in tqdm(models):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)

    hist = model.fit(generate_method(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generate_method(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

os.system("printf '\a'")

In [14]:
# Memory experiments on steering + throttle models

epochs = 8
batch_size = 32
verbose = 1

losses = []
val_losses = []
    
for memory in tqdm(memory_variants):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)
    
    generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer')
    frame_shape, numeric_shape, diff_shape = generator.get_shapes()
    tqdm.write('Target shape: {}'.format(diff_shape))
    tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))
    
    model = create_standalone_nvidia_cnn(activation='linear', input_shape=frame_shape, output_shape=diff_shape)
    #model = create_standalone_resnet(activation='linear', input_shape=frame_shape, output_shape=diff_shape)
    
    #mlp = create_mlp(input_shape=numeric_shape)
    #cnn = create_nvidia_cnn(input_shape=frame_shape)
    #model = create_multi_model_3(mlp, cnn, output_shape=diff_shape)

    hist = model.fit(generator.generate_with_numeric(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generator.generate_with_numeric(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()
    
os.system("printf '\a'")

n2_m1
Target shape: 2
Input shapes: (50, 180, 6); (4,)
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 4887 steps, validate for 1221 steps
Epoch 1/8
4887/4887 [==============================] - 282s 58ms/step - loss: 0.4775 - val_loss: 0.2794
Epoch 2/8
4887/4887 [==============================] - 289s 59ms/step - loss: 0.2524 - val_loss: 0.2391
Epoch 3/8
4887/4887 [==============================] - 291s 60ms/step - loss: 0.2297 - val_loss: 0.2237
Epoch 4/8
4887/4887 [==============================] - 290s 59ms/step - loss: 0.2203 - val_loss: 0.2221
Epoch 5/8
4887/4887 [==============================] - 290s 59ms/step - loss: 0.2146 - val_loss: 0.2158
Epoch 6/8
4887/4887 [==============================] - 291s 59ms/step - loss: 0.2110 - val_loss: 0.2146
Epoch 7/8
4887/4887 [==============================] - 291s 60ms/step - loss: 0.2082 - val_loss: 0.2087
Epoch 8/8
4887/4887 [==============================] - 290s 59ms/step - loss: 0.2065 - val_loss: 0.2101
Loss per epoch: [

4880/4880 [==============================] - 403s 83ms/step - loss: 0.2065 - val_loss: 0.2051
Epoch 7/8
4880/4880 [==============================] - 397s 81ms/step - loss: 0.2037 - val_loss: 0.2100
Epoch 8/8
4880/4880 [==============================] - 391s 80ms/step - loss: 0.2015 - val_loss: 0.2025
Loss per epoch: [0.447935483039769, 0.24383873771265394, 0.2239675339250291, 0.21573633328080177, 0.2103982123073007, 0.20650213071618412, 0.2036706636110168, 0.20151818241985117]
Validation loss per epoch: [0.263543716219605, 0.23143383977843113, 0.21934160360547364, 0.21230396382877084, 0.20432475946965764, 0.2051018223777169, 0.20996872025435087, 0.2024590887007166]



0

In [ ]:
# Memory experiments on gear models

epochs = 5
batch_size = 32
verbose = 1

gear_losses = []
gear_val_losses = []
    
for memory in tqdm(memory_variants):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)
    
    generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='all')
    frame_shape, numeric_shape, diff_shape = generator.get_shapes()
    tqdm.write('Target shape: {}'.format(diff_shape))
    tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))
    
    mlp = create_mlp(input_shape=numeric_shape)
    cnn = create_cnn_nvidia(input_shape=frame_shape)
    gear_model = create_multi_model_2(mlp, cnn, output_shape=2)

    hist = gear_model.fit(generator.generate_with_numeric(data='train'),
                          steps_per_epoch=generator.train_batch_count,
                          validation_data=generator.generate_with_numeric(data='test'),
                          validation_steps=generator.test_batch_count,
                          epochs=epochs, verbose=verbose)

    model_file_prefix = 'gear_model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(gear_model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    gear_model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    gear_losses.append(current_loss)
    gear_val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

In [ ]:
loss_data = []
val_loss_data = []

for i in range(0, len(losses)):
    val_loss_data.append({'data': val_losses[i], 'label': key, 'alpha': 1.0})
    loss_data.append({'data': losses[i], 'label': key, 'alpha': 1.0})
        
plot_stuff("val losses", val_loss_data, figsize=(10, 14))
plot_stuff("losses", loss_data, figsize=(10, 14))

In [ ]:
import tensorflow as tf

generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=32, column_mode='steer', separate_files=True)
frame_shape, numeric_shape, diff_shape = generator.get_shapes()

shapes = ([50, 180, 12], ())
types = (np.float32, np.float32)

train_dataset = tf.data.Dataset.from_generator(generator.generate_single_train, output_types=types, output_shapes=shapes)
train_dataset = train_dataset.batch(batch_size=32)
train_dataset = train_dataset.shuffle(buffer_size=10)

val_dataset = tf.data.Dataset.from_generator(generator.generate_single_test, output_types=types, output_shapes=shapes)
val_dataset = val_dataset.batch(batch_size=32)
val_dataset = val_dataset.shuffle(buffer_size=10)

model = create_cnn_alone(input_shape=frame_shape, output_shape=diff_shape)

hist = model.fit(train_dataset,
                 steps_per_epoch=generator.train_batch_count,
                 validation_data=val_dataset,
                 validation_steps=generator.test_batch_count,
                 epochs=10, verbose=1)